In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.contrib import slim

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [47]:
from PIL import Image

In [49]:
new_image = Image.new('RGB' , (52,52) , (128,128,128))

In [73]:
a=np.array([[1,2,3],[0,3,2],[2,3,4]])

In [74]:
a

array([[1, 2, 3],
       [0, 3, 2],
       [2, 3, 4]])

In [75]:
b = a[... , 0]>0

In [76]:
b

array([ True, False,  True])

In [77]:
a[1 , b[1]]

array([], shape=(0, 3), dtype=int32)

In [59]:
boolean = np.logical_and(np.array([12,3,1]) < 2 , np.array( [3,3,4] ) > 1)

In [78]:
c=np.array([1,2])

In [81]:
np.expand_dims(c , -2)

array([[1, 2]])

In [63]:
a[boolean]

array([[2, 3, 4]])

In [6]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense

In [9]:
_input = Input(shape=(784,))
output = Dense(10)(_input)

In [10]:
model = Model(_input , output)

In [14]:
model.input

<tf.Tensor 'input_2:0' shape=(?, 784) dtype=float32>

In [ ]:
model.fit_generator()

In [30]:
y_true = [Input(shape=( 416//{0:32, 1:16, 2:8}[l], 416//{0:32, 1:16, 2:8}[l], \
        3, 25)) for l in range(3)]

In [31]:
y_true

[<tf.Tensor 'input_3:0' shape=(?, 13, 13, 3, 25) dtype=float32>,
 <tf.Tensor 'input_4:0' shape=(?, 26, 26, 3, 25) dtype=float32>,
 <tf.Tensor 'input_5:0' shape=(?, 52, 52, 3, 25) dtype=float32>]

In [33]:
import keras.backend as K

In [43]:
_eval = K.switch(K.greater(tf.Variable([1,3])  , tf.Variable([2,4]) ) , [2,2] , [3,3])

AttributeError: 'list' object has no attribute 'get_shape'

In [46]:
with tf.Session() as sess:
    print( K.zeros_like([1,2,3]).eval() )

[0 0 0]


In [ ]:
tf.TensorArray()

In [43]:
with tf.Session() as sess:
    a=tf.range(13 , dtype=tf.float32)
    b=tf.range(13 , dtype=tf.float32)
    
    c1,c2=tf.meshgrid(a,b)
    
    x_offset = tf.reshape(c1,(-1,1))
    y_offset = tf.reshape(c2,(-1,1))
    print(x_offset.get_shape().as_list())
    print(y_offset.get_shape().as_list())

    
    x_y_offset = tf.concat([x_offset , y_offset] , axis=-1) #13*13  2
    x_y_offset = tf.reshape(tf.tile(x_y_offset , [1,3]) , [1,-1,2])
    print(x_y_offset.get_shape().as_list())
    
    test = sess.run(x_y_offset)

[169, 1]
[169, 1]
[1, 507, 2]


In [45]:
test[0]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       ...,
       [12., 12.],
       [12., 12.],
       [12., 12.]], dtype=float32)

In [38]:
13*13*3

507

In [35]:
test

[array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.]],
       dtype=float32),
 

In [46]:
def darknet53():
    """
    Builds Darknet-53 model.
    """
    inputs = tf.placeholder(dtype=tf.float32 , shape=[1 , 416 , 416 , 3])    
    
    inputs = _conv2d_fixed_padding(inputs, 32, 3)
    inputs = _conv2d_fixed_padding(inputs, 64, 3, strides=2)
    
    inputs = _darknet53_block(inputs, 32)
    
    
    inputs = _conv2d_fixed_padding(inputs, 128, 3, strides=2)

    for i in range(2):
        inputs = _darknet53_block(inputs, 64)

    inputs = _conv2d_fixed_padding(inputs, 256, 3, strides=2)

    for i in range(8):
        inputs = _darknet53_block(inputs, 128)
        
    route_1 = inputs
    
    inputs = _conv2d_fixed_padding(inputs, 512, 3, strides=2)

    for i in range(8):
        inputs = _darknet53_block(inputs, 256)

    route_2 = inputs
    
    inputs = _conv2d_fixed_padding(inputs, 1024, 3, strides=2)

    for i in range(4):
        inputs = _darknet53_block(inputs, 512)
        
    print(inputs.get_shape().as_list())
    # return route_1, route_2, inputs


def _conv2d_fixed_padding(inputs, filters, kernel_size, strides=1):
    if strides > 1:
        inputs = _fixed_padding(inputs, kernel_size)
    inputs = slim.conv2d(inputs, filters, kernel_size, stride=strides,
                         padding=('SAME' if strides == 1 else 'VALID'))
    return inputs


def _darknet53_block(inputs, filters):
    shortcut = inputs
    inputs = _conv2d_fixed_padding(inputs, filters , 1)
    
    inputs = _conv2d_fixed_padding(inputs, filters * 2 , 3)
    
    inputs = inputs + shortcut
    return inputs



def _fixed_padding(inputs, kernel_size, mode='CONSTANT'):
    
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg

    
    return tf.pad(inputs, [[0, 0], [pad_beg, pad_end], [pad_beg, pad_end], [0, 0]], mode=mode)


In [47]:
darknet53()

[1, 13, 13, 1024]
